In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask-ngrok

In [3]:
%cd /content/drive/MyDrive/Self \Case \Study \2/DATASET/Deployment

/content/drive/MyDrive/Self Case Study 2/DATASET/Deployment


In [4]:
import os
os.getcwd()

'/content/drive/MyDrive/Self Case Study 2/DATASET/Deployment'

In [5]:
from __future__ import division, print_function
# Flask utils
from flask_ngrok import run_with_ngrok
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
# 
import io
import os 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import numpy as np 
import time

try:
    print("Try.11.")
    from PIL import Image
except ImportError:
    print("Error")
    import Image

from build_image import read_image,classify_decode
from build_models import build_segment
from prediction import prediction 

Try.11.


In [6]:
# Define a flask 
app = Flask(__name__)

run_with_ngrok(app)

classify_weights_path = "/content/drive/MyDrive/Self Case Study 2/DATASET/chexnet_model_new.h5"
segment_weights_path = '/content/drive/MyDrive/Self Case Study 2/DATASET/best_Double_Unet.hdf5'

pred = prediction(classify_weights_path,segment_weights_path)

80142336/80134624 [==============================] - 1s 0us/step
(None, 64, 1, 1)
(None, 32, 1, 1)
(None, 32, 1, 1)
(None, 64, 1, 1)
(None, 256, 1, 1)
(None, 128, 1, 1)
Classification Model Loaded....
Segmentation Model Loaded...


In [7]:
#https://stackoverflow.com/questions/53202636/render-dynamically-changing-images-with-same-filenames-in-flask

@app.route('/', methods=['GET', 'POST']) 
def main_page():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join('uploaded_image', filename))
        return redirect(url_for('prediction', filename=filename))
    return render_template('index.html')


@app.route('/prediction/<filename>') 
def prediction(filename):

    # Make prediction
    image_path = filename
    
    classify_output, pred_mask = pred.Predict(image_path)
    print("Classification Output : ",classify_output)
    if(classify_output> 0.5):
        print("Classifier Prediction Confidence : {}%".format(classify_output*100))
        print('segment printing')
        pred_mask1 = np.squeeze(pred_mask[:,:,:,1])
        plt.imsave('predicted_mask.png', pred_mask1)

        background = Image.open(image_path)
        overlay = Image.open('predicted_mask.png')

        background = background.convert("RGBA")
        overlay = overlay.convert("RGBA")

        new_img = Image.blend(background, overlay, 0.3)

        new_graph_name = "Final_Output_pos_" + str(time.time()) + ".png"

        for filename in os.listdir('static/'):
            if filename.startswith('Final_Output_p'):  # not to remove other images
                os.remove('static/' + filename)

        new_img.save("/content/drive/MyDrive/Self Case Study 2/DATASET/Deployment/static/" + new_graph_name)

        classify_result = classify_output*100
        classify_text = 'Pneumothorax Found..!!'
        
    else:
        print('No Pneumothorax Detection...!')
        no_confidence = 1 - classify_output
        print('Classifier Prediction Confidence : {}%'.format(no_confidence*100))
        classify_result = no_confidence*100
        classify_text = 'No Pneumothorax Detection...!'
        background = Image.open(image_path)

        new_graph_name = "Final_Output_neg_" + str(time.time()) + ".png"

        for filename in os.listdir('static/'):
            if filename.startswith('Final_Output_n'):  # not to remove other images
                os.remove('static/' + filename)

        background.save("/content/drive/MyDrive/Self Case Study 2/DATASET/Deployment/static/" + new_graph_name)

        

    return render_template('base.html',segmented_image = new_graph_name, result = classify_result, review_text = classify_text)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b5255e6dee9e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Apr/2021 09:44:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 09:44:39] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Apr/2021 09:44:45] "POST / HTTP/1.1" 302 -


Pneumothorax Found..!!
Classifier Prediction Confidence : [[93.41909]]%
Classification Output :  [[0.93419087]]
Classifier Prediction Confidence : [[93.41909]]%
segment printing


...

127.0.0.1 - - [08/Apr/2021 09:47:33] "GET /static/Final_Output_pos_1617875252.4943774.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 09:47:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 09:47:51] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [08/Apr/2021 09:47:54] "GET /prediction/test_image_2.png HTTP/1.1" 200 -


Classification Output :  [[2.1266524e-07]]
No Pneumothorax Detection...!
Classifier Prediction Confidence : [[99.99998]]%


127.0.0.1 - - [08/Apr/2021 09:47:55] "GET /static/Final_Output_neg_1617875274.669552.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 09:48:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 09:48:19] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [08/Apr/2021 09:48:19] "GET /prediction/test_image_3.png HTTP/1.1" 200 -


Classification Output :  [[6.0890743e-05]]
No Pneumothorax Detection...!
Classifier Prediction Confidence : [[99.99391]]%


127.0.0.1 - - [08/Apr/2021 09:48:20] "GET /static/Final_Output_neg_1617875299.7004037.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 09:48:30] "GET / HTTP/1.1" 200 -


In [ ]:
for filename in os.listdir('static/'):
    if filename.startswith('Final_Output_1'):
        os.remove('static/' + filename)